In [13]:
import re

def load_data(is_test=False):
    if is_test:
        input_txt = "input_.txt"
    else:
        input_txt = "input.txt"
        
    with open(input_txt) as f:
        lines = f.read().strip().split('\n\n')

    seeds                = line2numbers(lines[0].replace('seeds: ', ''))
    seed2soil            = extract_mappings(lines[1], 'seed', 'soil')
    soil2fertilizer      = extract_mappings(lines[2], 'soil', 'fertilizer')
    fertilizer2water     = extract_mappings(lines[3], 'fertilizer', 'water')
    water2light          = extract_mappings(lines[4], 'water', 'light')
    light2temperature    = extract_mappings(lines[5], 'light', 'temperature')
    temperature2humidity = extract_mappings(lines[6], 'temperature', 'humidity')
    humidity2location    = extract_mappings(lines[7], 'humidity', 'location')

    return (seeds, seed2soil, soil2fertilizer, fertilizer2water, water2light, light2temperature, temperature2humidity, humidity2location)

In [35]:
def line2numbers(line):
    return [int(x) for x in re.findall('\d+', line)]


def extract_mappings(line, source, destination):
    line_ = line.replace(source + '-to-' + destination + ' map:\n', '')
    return [line2numbers(x) for x in line_.split('\n')]


def _convert(source, mapping):
    destination_begin, source_begin, range_length = mapping
    destination = source
    if (source >= source_begin) and (source < source_begin+range_length):
        destination += destination_begin - source_begin
    return destination


def convert(source, mappings):
    destination = source 
    for mapping in mappings: 
        #destination = _convert(destination, mapping)
        destination = _convert(source, mapping)
        if destination != source:
            break
    return destination


def seed2location(seed, data, debug=False):
    # this function can be written shortly using loop,
    # but for the debug purpose
    # just wrote down all the names.
    (seeds, 
     seed2soil, 
     soil2fertilizer, 
     fertilizer2water, 
     water2light, 
     light2temperature, 
     temperature2humidity, 
     humidity2location) = data
    
    soil        = convert(seed, seed2soil)
    fertilizer  = convert(soil, soil2fertilizer)
    water       = convert(fertilizer, fertilizer2water)
    light       = convert(water, water2light)
    temperature = convert(light, light2temperature)
    humidity    = convert(temperature, temperature2humidity)
    location    = convert(humidity, humidity2location)

    if debug:
        print(f"soil: {soil}")
        print(f"fertilizer: {fertilizer}")
        print(f"water: {water}")
        print(f"light: {light}")
        print(f"temperature: {temperature}")
        print(f"humidity: {humidity}")
        print(f"location: {location}")

    return location

In [38]:
## part 1
data = load_data(is_test=False)
(seeds, seed2soil, soil2fertilizer, fertilizer2water, water2light, light2temperature, temperature2humidity, humidity2location) = data
locations = [seed2location(seed, data) for seed in seeds]

#print(f"seeds: {seeds}")
#print(f"locations: {locations}")
print(f"the lowest location number that corresponds to any of the initial seed numbers: {min(locations)}")

the lowest location number that corresponds to any of the initial seed numbers: 51752125
